In [23]:
from transformers import pipeline

class EmotionAnalyzer:
    _model = None

    @classmethod
    def load_model(cls):
        if cls._model is None:
            cls._model = pipeline(
                "text-classification",
                model="j-hartmann/emotion-english-distilroberta-base",
                return_all_scores=True
            )
        return cls._model


In [24]:
response_text = "I’m stressed and it’s affecting my work"
emotion_model = EmotionAnalyzer.load_model()
model_output = emotion_model(response_text)[0]
print(model_output)

Device set to use cpu


[{'label': 'anger', 'score': 0.3341940939426422}, {'label': 'disgust', 'score': 0.014764036051928997}, {'label': 'fear', 'score': 0.03438035026192665}, {'label': 'joy', 'score': 0.002689182059839368}, {'label': 'neutral', 'score': 0.028367770835757256}, {'label': 'sadness', 'score': 0.577599823474884}, {'label': 'surprise', 'score': 0.008004691451787949}]


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [26]:
POSITIVE_EMOTIONS = ["joy"]
NEGATIVE_EMOTIONS = ["anger", "sadness", "fear", "disgust"]
NEUTRAL_EMOTIONS = ["neutral"]


def emotional_breakdown(model_output):
    positive, negative, neutral = 0.0, 0.0, 0.0

    for item in model_output:
        label = item["label"].lower()
        score = item["score"]

        if label in POSITIVE_EMOTIONS:
            positive += score
        elif label in NEGATIVE_EMOTIONS:
            negative += score
        elif label in NEUTRAL_EMOTIONS:
            neutral += score

    total = positive + negative + neutral

    if total == 0:
        return {"positive": 0, "negative": 0, "neutral": 0}

    return {
        "positive": round((positive / total) * 100, 2),
        "negative": round((negative / total) * 100, 2),
        "neutral": round((neutral / total) * 100, 2),
    }
breakdown = emotional_breakdown(model_output)
breakdown

{'positive': 0.27, 'negative': 96.87, 'neutral': 2.86}

In [8]:
EMOTION_WEIGHTS = {
    "positive": 1.0,
    "neutral": 0.6,
    "negative": 0.2
}


def calculate_eq_score(emotional_breakdown):
    score = (
        emotional_breakdown["positive"] * EMOTION_WEIGHTS["positive"] +
        emotional_breakdown["neutral"] * EMOTION_WEIGHTS["neutral"] +
        emotional_breakdown["negative"] * EMOTION_WEIGHTS["negative"]
    ) / 100

    return round(score * 100, 2)
eq_score = calculate_eq_score(breakdown)
eq_score

21.36

In [10]:
category_scores_raw={}
category = "emotion"
category_scores_raw.setdefault(category, []).append(eq_score)
category_scores_raw

{'emotion': [21.36]}

In [11]:
emotional_summary=[]
emotional_summary.append(breakdown)
emotional_summary

[{'positive': 0.27, 'negative': 96.87, 'neutral': 2.86}]

In [22]:
CATEGORY_EMOTION_WEIGHTS = {
    "emotional_regulation": {"negative": 0.1},
    "self_awareness": {"negative": 0.5}
}
# CATEGORY_EMOTION_WEIGHTS['self_awareness']['negative']
def finds(category):
  print(category)
  if category=="self_awareness":
      negative_weight = CATEGORY_EMOTION_WEIGHTS[category]['negative']
      print(negative_weight)

if __name__ == "__main__":
  category="self_awareness"
  finds(category)

self_awareness
0.5


In [27]:
EMOTION_WEIGHTS = {
    "positive": 1.0,
    "neutral": 0.6,
}

CATEGORY_EMOTION_WEIGHTS = {
    "emotional_regulation": {"negative": 0.1},
    "self_awareness": {"negative": 0.5}
}


def calculate_categorywise_EQ_score(category, emotional_breakdown):

    if category=="self_awareness":
        negative_weight = CATEGORY_EMOTION_WEIGHTS[category]['negative']
        score = (
            emotional_breakdown["positive"] * EMOTION_WEIGHTS["positive"] +
            emotional_breakdown["neutral"] * EMOTION_WEIGHTS["neutral"] +
            emotional_breakdown["negative"] * negative_weight
        ) / 100

        return round(score * 100, 2)

    elif category=="emotional_regulation":
        negative_weight = CATEGORY_EMOTION_WEIGHTS[category]['negative']
        score = (
            emotional_breakdown["positive"] * EMOTION_WEIGHTS["positive"] +
            emotional_breakdown["neutral"] * EMOTION_WEIGHTS["neutral"] +
            emotional_breakdown["negative"] * negative_weight
        ) / 100

        return round(score * 100, 2)
category="self_awareness"
eq_score = calculate_categorywise_EQ_score(category, breakdown)
eq_score

50.42

In [29]:
CATEGORY_EMOTION_WEIGHTS = {
    "self_awareness": {"negative": 0.5},
    "emotional_regulation": {"negative": 0.25},
    "empathy": {"negative": 0.4},
    "conflict_resolution": {"negative": 0.2},
    "motivation": {"negative": 0.2},
    "social_skills": {"negative": 0.2}
}

total = sum(v["negative"] for v in CATEGORY_EMOTION_WEIGHTS.values())
for k in CATEGORY_EMOTION_WEIGHTS:
    CATEGORY_EMOTION_WEIGHTS[k]["negative"] /= total
    print(CATEGORY_EMOTION_WEIGHTS)

{'self_awareness': {'negative': 0.2857142857142857}, 'emotional_regulation': {'negative': 0.25}, 'empathy': {'negative': 0.4}, 'conflict_resolution': {'negative': 0.2}, 'motivation': {'negative': 0.2}, 'social_skills': {'negative': 0.2}}
{'self_awareness': {'negative': 0.2857142857142857}, 'emotional_regulation': {'negative': 0.14285714285714285}, 'empathy': {'negative': 0.4}, 'conflict_resolution': {'negative': 0.2}, 'motivation': {'negative': 0.2}, 'social_skills': {'negative': 0.2}}
{'self_awareness': {'negative': 0.2857142857142857}, 'emotional_regulation': {'negative': 0.14285714285714285}, 'empathy': {'negative': 0.2285714285714286}, 'conflict_resolution': {'negative': 0.2}, 'motivation': {'negative': 0.2}, 'social_skills': {'negative': 0.2}}
{'self_awareness': {'negative': 0.2857142857142857}, 'emotional_regulation': {'negative': 0.14285714285714285}, 'empathy': {'negative': 0.2285714285714286}, 'conflict_resolution': {'negative': 0.1142857142857143}, 'motivation': {'negative': 

In [4]:
CATEGORY_EMOTION_WEIGHTS = {
    "self_awareness": {"negative": 0.5},
    "emotional_regulation": {"negative": 0.25},
    "empathy": {"negative": 0.4},
    "conflict_resolution": {"negative": 0.2},
    "motivation": {"negative": 0.2},
    "social_skills": {"negative": 0.2}
}
# Get category-specific negative weight
category="emotional_regulation"
negative_weight = CATEGORY_EMOTION_WEIGHTS.get(category, {}).get("negative", 0.2)
negative_weight


0.25